In [1]:
import sympy as sp
from sympy.interactive import printing
printing.init_printing(use_latex=True)

In [2]:
%matplotlib inline

In [3]:
x = sp.Symbol('x')
x

In [4]:
a, b, c, d = sp.symbols('y1__0 y_1__0 y_1^0 \\dot{m}_{1}')
a, b, c, d

In [5]:
sp.symbols('alpha beta gamma')

In [6]:
sp.symbols('x_:2__:3')

In [7]:
sp.symbols(('a:c', 'x:z'))

In [8]:
sp.symbols('x(:2)(:3)')

In [10]:
sp.symbols('x((:c))')

In [11]:
x._hashable_content()

('x', ('commutative', True))

In [14]:
((x+1)**2).equals(x**2 + 2*x + 1)

True

In [15]:
f = sp.Function('f')

In [17]:
f(x)

In [18]:
g = sp.Lambda(x, x**2 +3)
g

In [19]:
g(1)